In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.utils import resample

cleaned = pd.read_csv('../data/cleaned_train_data.csv')
cleaned_test = pd.read_csv('../data/cleaned_test_data.csv')
cleaned['C_SEV'].value_counts()

2    179714
1      2559
Name: C_SEV, dtype: int64

In [3]:
cleaned['C_SEV'].value_counts()

2    179714
1      2559
Name: C_SEV, dtype: int64

In [4]:
# Separate majority and minority classes
majority = cleaned[cleaned.C_SEV==2]
minority = cleaned[cleaned.C_SEV==1]

# downsample majority class
maj_downsampled = resample(majority, 
                                 replace=True,     # sample with replacement
                                 n_samples=2559, # to match majority class
                                 random_state=407) # reproducible results
 
# Combine majority class with upsampled minority class
resampled = pd.concat([minority, maj_downsampled])
 
# Display new class counts
resampled['C_SEV'].value_counts()

2    2559
1    2559
Name: C_SEV, dtype: int64

In [5]:
X_train = resampled.drop(['C_SEV'], axis = 1)
y_train = resampled['C_SEV']
X_test = cleaned_test.drop(['C_SEV'], axis = 1)
y_test = cleaned_test['C_SEV']

In [6]:
categorical_features = ['C_MNTH', 'C_WDAY', 'C_HOUR', 'C_RCFG', 'C_WTHR', 'C_RSUR', 'C_RALN',
       'C_TRAF', 'V_TYPE']
categorical_transformer = Pipeline(steps=[
                                          ('onehot', OneHotEncoder(handle_unknown='ignore'))
                                         ])

preprocessor = ColumnTransformer(
                                 transformers=[
                                    ('cat', categorical_transformer, categorical_features)
                                ])

In [ ]:
### parameter tuning 

In [7]:
# rf_params = [{
#     'classifier' : [RandomForestClassifier()],
#     'classifier__n_estimators' : list(range(10,101,10)),
#     'classifier__max_features' : list(range(6,32,5))}]

# rf_model = { 'Random Forest': RandomForestClassifier()}

In [8]:
# log_params = [{'classifier' : [LogisticRegression()],
#     'classifier__penalty' : ['l1', 'l2'],
#     'classifier__C' : np.logspace(-4, 4, 20),
#     'classifier__solver' : ['liblinear']}]

# log_model = { 'Logistic Regression': LogisticRegression()}

In [15]:
# pipe = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('classifier', LogisticRegression())])
# search = GridSearchCV(pipe, log_params,
#                       n_jobs=-1,
#                       scoring = "accuracy",
#                       cv = 5,
#                       refit = True)
# search.fit(X_train, y_train)
# tr_err = 1 - search.score(X_train, y_train)
# valid_err = 1 - search.score(X_test, y_test)
# results_dict['Logistic Regression'] = [round(tr_err,3), round(valid_err,3), search.best_params_]

In [14]:
# pipe = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('classifier', RandomForestClassifier())])
# search = GridSearchCV(pipe, rf_params,
#                       n_jobs=-1,
#                       scoring = "accuracy",
#                       cv = 5,
#                       refit = True)
# search.fit(X_train, y_train)
# tr_err = 1 - search.score(X_train, y_train)
# valid_err = 1 - search.score(X_test, y_test)
# results_dict['Random Forest'] = [round(tr_err,3), round(valid_err,3), search.best_params_]

In [21]:
# pipe = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('classifier', LogisticRegression())])

# search = GridSearchCV(pipe, log_params,
#                       n_jobs=-1,
#                       scoring = "accuracy",
#                       cv = 5,
#                       refit = True)
# search.fit(X_train, y_train)
# tr_err = 1 - search.score(X_train, y_train)
# valid_err = 1 - search.score(X_test, y_test)
# results_dict['Logistic Regression'] = [round(tr_err,3), round(valid_err,3), search.best_params_]

In [16]:
# results_dict

{'Logistic Regression': [0.295,
  0.31,
  {'classifier': LogisticRegression(C=4.281332398719396, class_weight=None, dual=False,
                      fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                      max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                      random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                      warm_start=False),
   'classifier__C': 4.281332398719396,
   'classifier__penalty': 'l2',
   'classifier__solver': 'liblinear'}],
 'Random Forest': [0.019,
  0.243,
  {'classifier': RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                          criterion='gini', max_depth=None, max_features=6,
                          max_leaf_nodes=None, max_samples=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estima

In [18]:
# results = pd.DataFrame(results_dict).T
# results.columns = ("Train Score", "Validation Score", "Best Parameters")
# results

,Train Score,Validation Score,Best Parameters
Logistic Regression,0.295,0.31,{'classifier': LogisticRegression(C=4.28133239...
Random Forest,0.019,0.243,{'classifier': RandomForestClassifier(bootstra...


In [34]:
rf_params = [{
    'classifier' : [RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                          criterion='gini', max_depth=None, max_features=6,
                          max_leaf_nodes=None, max_samples=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=90,
                          n_jobs=None, oob_score=False, random_state=None,
                          verbose=0, warm_start=False)],
    'classifier__n_estimators' : 90,
    'classifier__max_features' : 6}]

rf_model = { 'Random Forest': RandomForestClassifier()}

In [35]:
log_params = [{'classifier': LogisticRegression(C=4.281332398719396, class_weight=None, dual=False,
                      fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                      max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                      random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                      warm_start=False),
   'classifier__C': 4.281332398719396,
   'classifier__penalty': 'l2',
   'classifier__solver': 'liblinear'}]

log_model = { 'Logistic Regression': LogisticRegression()}

In [36]:
pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier())])
rf = RandomForestClassifier(pipe, rf_params)
rf.fit(X_train, y_train)
tr_err = 1 - rf.score(X_train, y_train)
valid_err = 1 - rf.score(X_test, y_test)
results_dict['Random Forest'] = [round(tr_err,3), round(valid_err,3), rf.best_params_]

ValueError: n_estimators must be an integer, got <class 'sklearn.pipeline.Pipeline'>.

In [31]:
results_dict = {}
predict = {}